In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

##### Volume constant

In [ ]:
V = 10e-12
NA = 6.023e23
# sima had V*NA*100e-9*10e-3
vna = V*NA*10e-9*10e-3
ivna = 1/vna

In [ ]:
### p is the ODE system vector p=[f1, f2, f3, f4, ..., f19]
#### 0  -> clb1
#### 1  -> clb3
#### 2  -> cdc20T
#### 3  -> cdc20
#### 4  -> clb4
#### 5  -> sp
#### 6  -> cdc5t
#### 7  -> cdc5a 
#### 8  -> ndd1t
#### 9  -> ndd1a
#### 10 -> hcm1
#### 11 -> ndt80
#### 12 -> sum1iIme2
#### 13 -> sum1iCdk1
#### 14 -> sum1iRC
#### 15 -> ama1p
#### 16 -> rc
#### 17 -> dsb
#### 18 -> ama1t

## Parameters

In [ ]:
#Synthesis and degradation of Clb1 :
kClb1s = 0.002
kClb1sp = 0.2
kClb1spp = 0.1
kClb1d = 0.1
kClb1dp = 0.2
kClb1dpp = 0.02
#NOTE : Decreasing Clb1 intrinsic decay rate widens Clb3 duration
kClb3s = 0.002
kClb3sp = 0.5
kClb3d = 0.2
kClb3Cdc20d = 0.2
#Cdc20
kCdc20s = 0.2
kCdc20d = 0.1
kCdc20Clb1p = 0.1
kCdc20Clb3p = 0.1
kCdc20a = 0.1
JCdc20Clb3 = 0.1
JCdc20clb1 = 0.1
JCdc20 = 0.1
kClb1Cdc20d = 0.3
kClb3Cdc20d = 0.3
#Clb1 phosophorylation of Cdc20 weaker than Clb3
#Synthesis and degradation of Clb4 :
kClb4s = 0.2
kClb4sp = 0.1
kClb4d = 0.2
kClb4dp = 1
kClb4dpp = 0.02
#Activation and inactivation of SP :
kSPa = 2
kSPi = 2
JSP = 0.01
#Synthesis, degradation, activation, and inactivation of Cdc5 :
kCdc5s = 0.004
kCdc5sp = 0.03
kCdc5spp = 0.02
kCdc5d = 0.02
kCdc5dp = 0.06
kCdc5dpp = 0.002
kCdc5a = 0.1
kCdc5ap = 0.4
kCdc5app = 0.3
kCdc5i = 0.1
#Synthesis, degradation, activation, and inacti - vation of Ndd1 :
kNdd1s = 0.03
kNdd1d = 0.0001
kNdd1dp = 1
kNdd1dpp = 0.02
kNdd1a = 0.1
kNdd1ap = 0.2
kNdd1app = 0.04
kNdd1i = 0.2
JNdd1 = 0.04
#Degradation of Hcm1 :
kHcm1d = 0.02
#Regulation of Ndt80 :
kNdt80s = 0.01
kNdt80sp = 2
kNdt80d = 0.3
JNdt80p = 0.2
alpha = 1
beta = 0.1
ki = 0.01
#degradation of Ndt80 by Ama1 :
kNdt80dp = 0.6
#Regulation of Sum1 :
kSum1i = 0.025
kSum1a = 0.000001
kSum1ip = 0.1
kSum1ipp = 1
kSum1ap = 0.01
kSum1ippp = 0.25
kSum1app = 1
#Regulation of Ama1 :
kAma1a = 0.1
kAma1i = 0.0
kAma1ip = 0.1
JAma1 = 0.1
kAIas = 10
kAIds = 1
kAma1clb3p = 0.1
#Faster rate of Ama1 phosphorylation by Clb1
#Synthesis and degradation of the additional Ama1 - inhibitor (AI) :
kAIs = 0.1
kAId = 0.15
#Activation and inactivation of the RC :
kRCa = 1
kRCi = 0.1
kRCip = 2
JRC = 0.01
#Repair of DSBs :
kDSBi = 0.02
kdmRNA = 0.1
kRim4mRNA = 100
kAma1dp = 0.01
kAma1exp = 0.08
kAma1s = 0.01

## Initial conditions

In [ ]:
names = ['clb1','clb3','cdc20T','cdc20','clb4','sp','cdc5t','cdc5a','ndd1t','ndd1a','hcm1','ndt80','sum1iIme2','sum1iCdk1','sum1iRC','ama1p','rc','dsb','ama1t']
p = [0,0,0,0,0,0,0,0,0,0,vna,0,0,0,0,0,0,vna,vna]

proteinCount = {}
for i in range(0,len(names)): proteinCount[names[i]] = p[i]

proteinMap = {
    tuple(np.arange(0,5)): names[0],
    tuple(np.arange(5,10)): names[1],
    tuple(np.arange(10,12)): names[2],
    tuple(np.arange(12,18)): names[3],
    tuple(np.arange(18,22)): names[4],
    tuple(np.arange(22,27)): names[5],
    tuple(np.arange(27,31)): names[6],
    tuple(np.arange(31,40)): names[7],
    # constant rates of change for protiens 8,9
    tuple(np.arange(40,41)): names[10],
    tuple(np.arange(41,45)): names[11],
    tuple(np.arange(45,48)): names[12],
    tuple(np.arange(48,55)): names[13],
    tuple(np.arange(55,58)): names[14],
    tuple(np.arange(58,65)): names[15],
    tuple(np.arange(65,69)): names[16],
    tuple(np.arange(69,70)): names[17],
    tuple(np.arange(70,74)): names[18]
}   

In [ ]:
def gillespie_process(T,p):     
    # current time
    t = 0; treport=100
    
    # histories
    times = [t]
    proteins = [p]
    
    while t<T:
        
        # time functions
        kDSBi = 0.02
        Sum1T = 1*vna
        Dmc1 = 1*vna
        Sum1I = lambda t: p[12]*p[13]*p[14]/(Sum1T * Sum1T)
        JNdt80 = lambda t: JNdt80p*(vna+(alpha*(Sum1T-p[12])+beta*(p[12]-Sum1I(t)))/ki)
        Ama1 = lambda t: p[18]-p[15] 
        Rim4 = lambda t: (1-math.tanh(0.2*(t-240)))/2
        
        # propoensities
        # eq1
        p0 = kClb1s*vna
        p1 = kClb1sp*p[11]
        p2 = -kClb1d*p[0]
        p3 = -kClb1dp*Ama1(t)*p[0]*ivna
        p4 = -kClb1Cdc20d*p[3]*p[0]*ivna
        # eq2
        p5 = kClb3s*vna
        p6 = -kClb3d*p[1]
        p7 = -kClb3Cdc20d*p[3]*p[1]*ivna
        p8 = 5*kClb3sp*math.exp(-(t-240)/25)*vna
        p9 = -5*kClb3sp*math.exp(-(t-240)/25)*Rim4(t)*vna
        # eq3
        p10 = kCdc20s*vna
        p11 = -kCdc20d*p[2]
        # eq4
        p12 = (kCdc20Clb1p*p[0]*p[2])/(JCdc20clb1*vna+p[2]-p[3])
        p13 = (-kCdc20Clb1p*p[0]*p[3])/(JCdc20clb1*vna+p[2]-p[3])
        p14 = (kCdc20Clb3p*p[1]*p[2])/(JCdc20Clb3*vna+p[2]-p[3])
        p15 = (-kCdc20Clb3p*p[1]*p[3])/(JCdc20Clb3*vna+p[2]-p[3])
        p16 = -kCdc20a*p[3]*vna/(JCdc20*vna+p[3])
        p17 = -kCdc20d*p[3]
        # eq5
        p18 = kClb4s*vna
        p19 = kClb4sp*p[11]
        p20 = -kClb4d*p[4]
        p21 = -kClb4dp*Ama1(t)*p[4]*ivna
        # eq6
        p22 = (kSPa*vna*p[0])/(JSP*vna+vna-p[5])
        p23 = (kSPa*vna*p[4])/(JSP*vna+vna-p[5])
        p24 = -(kSPa*p[0]*p[5])/(JSP*vna+vna-p[5])
        p25 = -(kSPa*p[0]*p[5])/(JSP*vna+vna-p[5])
        p26 = -kSPi*vna*p[5]/(JSP*vna+p[5])
        # eq7
        p27 = kCdc5s*vna
        p28 = kCdc5sp*p[11]
        p29 = kCdc5d*p[6]
        p30 = kCdc5dp*Ama1(t)*p[6]*ivna
        # eq8
        p31 = kCdc5a*p[6]
        p32 = kCdc5ap*p[0]*p[6]*ivna
        p33 = kCdc5app*p[4]*p[6]*ivna
        p34 = -kCdc5a*p[7]
        p35 = -kCdc5ap*p[0]*p[7]*ivna
        p36 = -kCdc5app*p[4]*p[7]*ivna
        p37 = -kCdc5i*p[7]
        p38 = -kCdc5d*p[7]
        p39 = -kCdc5dp*Ama1(t)*p[7]*ivna
        # eq9/eq10 none
        # eq11
        p40 = -kHcm1d*p[10]
        # eq12
        p41 = kNdt80s*vna
        p42 = kNdt80sp*vna*p[11]/(JNdt80(t)+p[11])
        p43 = -kNdt80d*p[11]
        p44 = -kNdt80dp*Ama1(t)*p[11]*ivna
        # eq13
        p45 = kSum1i*Sum1T
        p46 = -kSum1i*p[12]
        p47 = -kSum1a*p[12]
        # eq14
        p48 = kSum1ip*Sum1T 
        p49 = kSum1ipp*Sum1T*p[0]*ivna
        p50 = kSum1ipp*Sum1T*p[4]*ivna
        p51 = -kSum1ip*p[13]
        p52 = -kSum1ipp*p[13]*p[0]*ivna
        p53 = -kSum1ipp*p[13]*p[4]*ivna
        p54 = -kSum1ap*p[13]
        # eq15
        p55 = kSum1ippp*Sum1T
        p56 = -kSum1ippp*p[14]
        p57 = -kSum1app*p[16]*p[14]*ivna
        # eq16
        p58 = (kAma1i*p[18]*vna)/(JAma1*vna+p[18]-p[15])
        p59 = (kAma1ip*p[0]*p[18]*ivna*vna)/(JAma1*vna+p[18]-p[15])
        p60 = (-kAma1i*vna*p[15])/(JAma1*vna+p[18]-p[15])
        p61 = (-kAma1ip*p[0]*p[15]*ivna*vna)/(JAma1*vna+p[18]-p[15])
        p62 = kAma1a*vna*p[15]/(JAma1*vna+p[15])
        p63 = (kAma1clb3p*p[1]*p[18]*ivna*vna)/(JAma1*vna+p[18]-p[15])
        p64 = (-kAma1clb3p*p[1]*p[15]*ivna*vna)/(JAma1*vna+p[18]-p[15])
        # eq17
        p65 = (kRCa*vna*p[17])/(JRC*vna+vna-p[16])
        p66 = (-kRCa*p[17]*p[16])/(JRC*vna+vna-p[16])
        p67 = (-kRCi*vna*p[16])/(JRC*vna+p[16])
        p68 = (-kRCip*p[7]*p[16]*ivna*vna)/(JRC*vna+p[16])
        # eq18
        p69 = -kDSBi*p[17]*Dmc1*ivna
        # eq19 **check**
        p70 = kAma1s*vna
        p71 = -kAma1dp*p[18]
        p72 = 40*kAma1s*math.exp(-(t-240)/100)*vna
        p73 = -40*kAma1s*Rim4(t)*math.exp(-(t-240)/100)*vna
        
        # ductape (im so sorry) we wanted abs(n)/n (my fault)
        birthDeathMap = []
        for i in range(0,74):
            sign = str(eval(f'p{i}'))[0]
            if sign == '-': birthDeathMap.append(-1)
            else: birthDeathMap.append(1)
        
        # sum propensities
        props = []
        tot = 0
        for i in range(0,74):
            props.append(abs(eval(f'p{i}')))
            #tot += abs(eval(f'p{i}'))
        
        props = np.array(props)
        
        # min max normalize prop values
        # props = (props-min(props))/(max(props)-min(props))

        tot = props.sum(axis=0)
         
        partial_sums = []
        for i in range(0,74):
            s = 0
            for j in range(0,i+1):
                s += abs(eval(f'p{j}'))
            partial_sums.append(s)
        
        probs = np.array(partial_sums)/tot        

        #change time
        t = t - np.log(np.random.random())/tot
        #print(t)
        
        # get a random number
        r = np.random.random()
        
        # get index where r falls in probability ranges
        index = -1
        for i in range(0,74):
            if r <= probs[i]:
                index = i
                break        
        
        for interval in proteinMap.keys():
            if index in interval:
                if proteinCount[proteinMap[interval]] == 0 and birthDeathMap[index] == -1:
                    pass
                else:
                    proteinCount[proteinMap[interval]] += birthDeathMap[index]
        
        # record Gillespie steps
        times.append(t)
        p = []
        for key in proteinCount.keys(): p.append(proteinCount[key])
        proteins.append(p)
        
        plt.close()
        for j in range(0, 19):
            points = []
            for i in range(0, len(proteins)):
                points.append(proteins[i][j])
            plt.plot(times, points)
        plt.show()
        
        #print(birthDeathMap)
    return times, proteins

In [ ]:
times, proteins = gillespie_process(660,p)

In [ ]:
for j in range(0, 19):
    points = []
    for i in range(0, len(proteins)):
        points.append(proteins[i][j])
    plt.plot(times, points)
plt.show()